In [1]:
import numpy as np
import pandas as pd
from scipy.io import mmread
import os
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6, 4)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import anndata as ad
import scanpy as sc
import maxfuse as mf
from tqdm import tqdm

2024-05-20 14:11:05.557254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 14:11:05.706513: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 14:11:05.710343: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/twang15/alleleCount-4.2.1/lib/
2024-05-20 14:11:05.710357: I tensorflow/compi

In [2]:
rna_adata = sc.read_h5ad("/data/twang15/spatial_protein/CODEX_data/hubmap/scRNA/large_intestine/large_intestine.h5ad")

In [3]:
rna_adata.obs

,assay_ontology_term_id,donor_id,anatomical_information,n_counts_UMIs,n_genes,cell_ontology_class,free_annotation,manually_annotated,compartment,sex_ontology_term_id,...,tissue_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,observation_joinid
cell_id,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGCGCCTTG_TSP14_LI_Distal_10X_1_1,EFO:0009922,TSP14,Distal,4572.0,1490,"cd4-positive, alpha-beta t cell",cd4-positive alpha-beta T cell,True,immune,PATO:0000384,...,tissue,"CD4-positive, alpha-beta T cell",10x 3' v3,normal,Homo sapiens,male,large intestine,European,59-year-old human stage,=?9TSos%=i
AAACCCACACTTGGGC_TSP14_LI_Distal_10X_1_1,EFO:0009922,TSP14,Distal,12874.0,1466,enterocyte of epithelium of large intestine,immature enterocyte,True,epithelial,PATO:0000384,...,tissue,enterocyte of epithelium of large intestine,10x 3' v3,normal,Homo sapiens,male,large intestine,European,59-year-old human stage,@%;bf?*BFM
AAACCCACATTCAGCA_TSP14_LI_Distal_10X_1_1,EFO:0009922,TSP14,Distal,4400.0,1666,monocyte,monocyte,True,immune,PATO:0000384,...,tissue,monocyte,10x 3' v3,normal,Homo sapiens,male,large intestine,European,59-year-old human stage,*^tPAZZp(j
AAACCCAGTATCTTCT_TSP14_LI_Distal_10X_1_1,EFO:0009922,TSP14,Distal,13988.0,2319,enterocyte of epithelium of large intestine,immature enterocyte,True,epithelial,PATO:0000384,...,tissue,enterocyte of epithelium of large intestine,10x 3' v3,normal,Homo sapiens,male,large intestine,European,59-year-old human stage,uLV;rz?W@i
AAACCCAGTGGTATGG_TSP14_LI_Distal_10X_1_1,EFO:0009922,TSP14,Distal,6641.0,1372,enterocyte of epithelium of large intestine,immature enterocyte,True,epithelial,PATO:0000384,...,tissue,enterocyte of epithelium of large intestine,10x 3' v3,normal,Homo sapiens,male,large intestine,European,59-year-old human stage,BbZFljt>80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSP2_LI_proximal_SS2_B114618_B134450_Stromal_P21_S381,EFO:0008931,TSP2,proximal,183430.0,1015,goblet cell,goblet cell,True,epithelial,PATO:0000383,...,tissue,large intestine goblet cell,Smart-seq2,normal,Homo sapiens,female,large intestine,African American or Afro-Caribbean,61-year-old human stage,p6HnaCnt(?
TSP2_LI_proximal_SS2_B114618_B134450_Stromal_P22_S382,EFO:0008931,TSP2,proximal,61756.0,672,intestinal tuft cell,intestinal tuft cell,True,epithelial,PATO:0000383,...,tissue,intestinal tuft cell,Smart-seq2,normal,Homo sapiens,female,large intestine,African American or Afro-Caribbean,61-year-old human stage,0+UFL@Uj*#
TSP2_LI_proximal_SS2_B114618_B134450_Stromal_P23_S383,EFO:0008931,TSP2,proximal,164072.0,1377,goblet cell,goblet cell,True,epithelial,PATO:0000383,...,tissue,large intestine goblet cell,Smart-seq2,normal,Homo sapiens,female,large intestine,African American or Afro-Caribbean,61-year-old human stage,U7n6*(2t2d


In [3]:
protein_adata = sc.read_h5ad("/data/twang15/spatial_protein/CODEX_data/hubmap/anndata/large_intestine/combined_protein.h5ad")

/data/twang15/mamba/envs/imc/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
rna_adata.var_names = rna_adata.var['feature_name']

/data/twang15/mamba/envs/imc/lib/python3.8/site-packages/anndata/_core/anndata.py:887: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['DDX11L1', 'WASH7P', 'MIR6859-1', 'MIR1302-2HG', 'MIR1302-2']

    Inferred to be: categorical

  names = self._prep_dim_index(names, "var")


In [7]:
correspondence = pd.read_csv("/data/twang15/maxfuse/docs/protein_gene_conversion.csv")
rna_protein_correspondence = []

for i in range(correspondence.shape[0]):
    curr_protein_name, curr_rna_names = correspondence.iloc[i]
    if curr_protein_name not in protein_adata.var_names:
        continue
    if curr_rna_names.find('Ignore') != -1: # some correspondence ignored eg. protein isoform to one gene
        continue
    curr_rna_names = curr_rna_names.split('/') # eg. one protein to multiple genes
    for r in curr_rna_names:
        if r in rna_adata.var_names:
            rna_protein_correspondence.append([r, curr_protein_name])
            

rna_protein_correspondence = np.array(rna_protein_correspondence)
_, unique_indices = np.unique(rna_protein_correspondence[:, 1], return_index=True)
rna_protein_correspondence = rna_protein_correspondence[sorted(unique_indices)]
_, unique_indices = np.unique(rna_protein_correspondence[:, 0], return_index=True)
rna_protein_correspondence = rna_protein_correspondence[sorted(unique_indices)]


protein_adata.obs_names_make_unique
rna_adata.obs_names_make_unique
protein_adata.var_names_make_unique
rna_adata.var_names_make_unique
mask1 = rna_adata.var_names.isin(rna_protein_correspondence[:, 0])
mask2 = protein_adata.var_names.isin(rna_protein_correspondence[:, 1])

rna_shared = rna_adata[:, mask1].copy()
# rna_shared = rna_adata.loc[:, rna_protein_correspondence[:, 0]].copy()
protein_shared = protein_adata[:, mask2].copy() 

mask = (
    (rna_shared.X.toarray().std(axis=0) > 1e-5) 
    & (protein_shared.X.std(axis=0) > 1e-5)
)
rna_shared = rna_shared[:, mask].copy()
protein_shared = protein_shared[:, mask].copy()

/data/twang15/mamba/envs/imc/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [8]:
# row sum for RNA
rna_counts = rna_shared.X.toarray().sum(axis=1)
# row sum for protein
protein_counts = protein_shared.X.sum(axis=1)
# take median of each and then take mean
target_sum = (np.median(rna_counts.copy()) + np.median(protein_counts.copy())) / 2


# process rna_shared
sc.pp.normalize_total(rna_shared, target_sum=target_sum)
sc.pp.log1p(rna_shared)
sc.pp.scale(rna_shared)

# sc.pp.neighbors(rna_shared, n_neighbors=15)
# sc.tl.umap(rna_shared)
# sc.pl.umap(rna_shared, color=['celltype.l1','celltype.l2'])

rna_shared = rna_shared.X.copy()

# process protein_shared
sc.pp.normalize_total(protein_shared, target_sum=target_sum)
sc.pp.log1p(protein_shared)
sc.pp.scale(protein_shared)

protein_shared = protein_shared.X.copy()

# process all RNA features
sc.pp.normalize_total(rna_adata)
sc.pp.log1p(rna_adata)
sc.pp.highly_variable_genes(rna_adata)
# only retain highly variable genes
# rna_adata = rna_adata[:, rna_adata.var.highly_variable].copy()
sc.pp.scale(rna_adata)


# process all protein features
sc.pp.normalize_total(protein_adata)
sc.pp.log1p(protein_adata)
sc.pp.scale(protein_adata)


# make sure no feature is static
rna_active = rna_adata.X
protein_active = protein_adata.X
rna_active = rna_active[:, rna_active.std(axis=0) > 1e-5]
protein_active = protein_active[:, protein_active.std(axis=0) > 1e-5]
# inspect shape of the four matrices
print(rna_active.shape)
print(protein_active.shape)
print(rna_shared.shape)
print(protein_shared.shape)

/data/twang15/mamba/envs/imc/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:196: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))


(13680, 47778)
(1027537, 24)
(13680, 22)
(1027537, 22)


In [9]:
fusor = mf.model.Fusor(
    shared_arr1=rna_shared,
    shared_arr2=protein_shared,
    active_arr1=rna_active,
    active_arr2=protein_active,
    labels1=None,
    labels2=None
)
fusor.split_into_batches(
    max_outward_size=10000,
    matching_ratio=4,
    metacell_size=2,
    verbose=True
)


The first data is split into 1 batches, average batch size is 13680, and max batch size is 13680.
The second data is split into 25 batches, average batch size is 41101, and max batch size is 41113.
Batch to batch correspondence is:
  ['0<->0', '0<->1', '0<->2', '0<->3', '0<->4', '0<->5', '0<->6', '0<->7', '0<->8', '0<->9', '0<->10', '0<->11', '0<->12', '0<->13', '0<->14', '0<->15', '0<->16', '0<->17', '0<->18', '0<->19', '0<->20', '0<->21', '0<->22', '0<->23', '0<->24'].


In [10]:
fusor.construct_graphs(
    n_neighbors1=15,
    n_neighbors2=15,
    svd_components1=10,
    svd_components2=10,
    resolution1=2,
    resolution2=2,
    randomized_svd=False,
    svd_runs=1,
    resolution_tol=0.1,
    leiden_runs=1,
    leiden_seed=None,
    verbose=True
)

Aggregating cells in arr1 into metacells of average size 2...
Constructing neighborhood graphs for cells in arr1...
Now at batch 0...
Graph construction finished!
Clustering into metacells...
Now at batch 0...
Metacell clustering finished!
Constructing neighborhood graphs for cells in arr1...
Now at batch 0...
Graph construction finished!
Clustering the graphs for cells in arr1...
Now at batch 0...
Graph clustering finished!
Constructing neighborhood graphs for cells in arr2...
Now at batch 0...
Now at batch 1...
Now at batch 2...
Now at batch 3...
Now at batch 4...
Now at batch 5...
Now at batch 6...
Now at batch 7...
Now at batch 8...
Now at batch 9...
Now at batch 10...
Now at batch 11...
Now at batch 12...
Now at batch 13...
Now at batch 14...
Now at batch 15...
Now at batch 16...
Now at batch 17...
Now at batch 18...
Now at batch 19...
Now at batch 20...
Now at batch 21...
Now at batch 22...
Now at batch 23...
Now at batch 24...
Graph construction finished!
Clustering the graphs f

In [11]:
fusor.find_initial_pivots(
    wt1=0.7, wt2=0.7,
    svd_components1=15, svd_components2=10
)

Now at batch 0<->0...
Now at batch 0<->1...
Now at batch 0<->2...
Now at batch 0<->3...
Now at batch 0<->4...
Now at batch 0<->5...
Now at batch 0<->6...
Now at batch 0<->7...
Now at batch 0<->8...
Now at batch 0<->9...
Now at batch 0<->10...
Now at batch 0<->11...
Now at batch 0<->12...
Now at batch 0<->13...
Now at batch 0<->14...
Now at batch 0<->15...
Now at batch 0<->16...
Now at batch 0<->17...
Now at batch 0<->18...
Now at batch 0<->19...
Now at batch 0<->20...
Now at batch 0<->21...
Now at batch 0<->22...
Now at batch 0<->23...
Now at batch 0<->24...
Done!


In [12]:
fusor.refine_pivots(
    wt1=0.3, wt2=0.3,
    svd_components1=40, svd_components2=None,
    cca_components=20,
    n_iters=1,
    randomized_svd=False,
    svd_runs=1,
    verbose=True
)




fusor.filter_bad_matches(target='pivot', filter_prop=0.05)



pivot_matching = fusor.get_matching(target='pivot')



fusor.propagate(
    svd_components1=40,
    svd_components2=None,
    wt1=0.7,
    wt2=0.7,
)
fusor.filter_bad_matches(
    target='propagated',
    filter_prop=0.3
)
full_matching = fusor.get_matching(order=(2, 1), target='full_data')


Now at batch 0<->0...
Now at batch 0<->1...
Now at batch 0<->2...
Now at batch 0<->3...
Now at batch 0<->4...
Now at batch 0<->5...
Now at batch 0<->6...
Now at batch 0<->7...
Now at batch 0<->8...
Now at batch 0<->9...
Now at batch 0<->10...
Now at batch 0<->11...
Now at batch 0<->12...
Now at batch 0<->13...
Now at batch 0<->14...
Now at batch 0<->15...
Now at batch 0<->16...
Now at batch 0<->17...
Now at batch 0<->18...
Now at batch 0<->19...
Now at batch 0<->20...
Now at batch 0<->21...
Now at batch 0<->22...
Now at batch 0<->23...
Now at batch 0<->24...
Done!
Begin filtering...
Now at batch 0<->0...
Now at batch 0<->1...
Now at batch 0<->2...
Now at batch 0<->3...
Now at batch 0<->4...
Now at batch 0<->5...
Now at batch 0<->6...
Now at batch 0<->7...
Now at batch 0<->8...
Now at batch 0<->9...
Now at batch 0<->10...
Now at batch 0<->11...
Now at batch 0<->12...
Now at batch 0<->13...
Now at batch 0<->14...
Now at batch 0<->15...
Now at batch 0<->16...
Now at batch 0<->17...
Now at

In [13]:
matching = pd.DataFrame(list(zip(full_matching[0], full_matching[1], full_matching[2])), columns = ['mod1_indx', 'mod2_indx', 'score'])

In [14]:
matching.to_csv('/data/twang15/spatial_protein/CODEX_data/hubmap/integrated/large_intestine/matching.tsv', sep='\t')

In [11]:
protein_inte = protein_adata[full_matching[1]].copy()

rna_adata_for_integrated = rna_adata[full_matching[0]]

protein_integrated = rna_adata_for_integrated.copy()
protein_inte.obs_names = protein_integrated.obs_names
# protein_integrated.obs = protein_inte.obs
protein_integrated.obs = pd.concat([protein_integrated.obs, protein_inte.obs], axis=1)

print(protein_integrated.obs)
print(protein_integrated.var)


/data/twang15/mamba/envs/imc/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/imc/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


                                                   assay_ontology_term_id  \
cell_id                                                                     
AGTCACACAAGCAGGT_TSP14_LI_Distal_10X_1_1                      EFO:0009922   
CCTAACCAGAAAGTCT_TSP14_LI_Distal_10X_1_1                      EFO:0009922   
TTCTCTCCAAATGGCG_TSP14_LI_Distal_10X_1_1                      EFO:0009922   
CAACAGTGTAGTTAGA_TSP14_LI_Proximal_10X_1_1                    EFO:0009922   
TTACTGTTCGTTCTAT_TSP14_LI_Distal_10X_1_1                      EFO:0009922   
...                                                                   ...   
ACCCTTGCATACAGAA_TSP2_LI_distal_10X_1_1_Sigmoid               EFO:0009922   
CGTAGTAGTTGGTAGG_TSP2_LI_distal_10X_1_1_Sigmoid               EFO:0009922   
GGGAAGTCAATTTCTC_TSP2_LI_proximal_10X_1_1_Ascen...            EFO:0009922   
TSP2_LI_proximal_SS2_B114586_B134454_Epithelial...            EFO:0008931   
GTAGAAACACGAAAGC_TSP2_LI_distal_10X_1_1_Sigmoid               EFO:0009922   

In [12]:
protein_integrated.write_h5ad('/data/twang15/spatial_protein/CODEX_data/hubmap/integrated/large_intestine/combined_protein_integrated.h5ad')

In [10]:
full_matching = pd.read_csv('/data/twang15/spatial_protein/CODEX_data/hubmap/integrated/large_intestine/matching.tsv', sep='\t', index_col='Unnamed: 0')

In [8]:
matching

,mod1_indx,mod2_indx,score
0,1282,8,0.560078
1,2695,9,0.258774
2,7345,13,0.680557
3,8574,15,0.512414
4,7168,17,0.387523
...,...,...,...
772321,11521,1027532,0.751603
772322,11846,1027533,0.301009
772323,12944,1027534,0.406181
772324,13529,1027535,0.601993
